<a href="https://colab.research.google.com/github/DrumDong/Donggyu/blob/master/kaggle%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%97%B0%EC%8A%B5/%ED%85%8D%EC%8A%A4%ED%8A%B8%EB%B6%84%EB%A5%98_20%EB%89%B4%EC%8A%A4%EA%B7%B8%EB%A3%B9_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset='all',random_state=156)

In [4]:
print(news_data.keys()) # key값 확인

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


Target 클래스가 어떻게 구성돼 있는지 확인해 보자.

In [5]:
import pandas as pd

print('target 클래스의 값과 분포도 \n',pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n', news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [6]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

데이터를 다시 내려받고,헤더와 푸터 정보들을 삭제하자 (class값과 비슷한 것들이 많으므로 ) -> 순수한 텍스트만을 사용해 분석을 해야함

In [7]:
from sklearn.datasets import fetch_20newsgroups

# subset='train'으로 학습용 데이터만 추출, remove=('header','footer','quotes')로 내용만 추출
train_news = fetch_20newsgroups(subset='train',remove=('header','footer','quotes'),
                                random_state=156)

X_train = train_news.data
y_train = train_news.target

# subset='target'으로 학습용 데이터만 추출, remove=('header','footer','quotes')로 내용만 추출
test_news = fetch_20newsgroups(subset='test',remove=('header','footer','quotes'),
                   random_state=156)
X_test = test_news.data
X_target = test_news.target
print('학습 데이터 크기{0}, 테스트 데이터 크기{1}'.format(len(train_news.data),
                                           len(test_news.data)))

학습 데이터 크기11314, 테스트 데이터 크기7532


# 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가

## CountVectorizer

CountVectorizer를 이용해 학습 데이터의 텍스트를 피처 백터화 시키자.
- 테스트 데이터에서 CountVectorizer를 적용할 때는 반드시 학습 데이터를 이용해 fit()이 수행된 CountVectorizer 객체를 이용해 테스트 데이터를 변환(transform)해야 한다는 것 (cnt_vect.transform) 사용


In [8]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 피처 백터화 변환 수행
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train,y_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit()된 CountVectorizer를 이용해 테스트 데이터를 피터 벡처화 변환 수행.
X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 텍스트의 CountVectorzier Shape:',X_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorzier Shape: (11314, 120756)


 11314개의 문서에서 120756개의 단어
 
 ---

 로지스틱 회귀를 적용해 분류를 예측해 보자

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# LogisticRegression을 이용해 학습/예측/평가 수행.
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect,y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(X_target,
                                                                                   pred)))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


CountVectorized Logistic Regression의 예측 정확도는 0.769


## TF-IDF 기반으로 벡터화를 변경해 예측 모델 수행

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF_IDF 벡터화를 적용해 학습 데이터 세트와 테스트 데이터 세트 변환.
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train,y_train)
X_train_tfidf_vect =tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# LogisticRegression을 이용해 학습/예측/평가 수행.
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect,y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(X_target,
                                                                                   pred)))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


CountVectorized Logistic Regression의 예측 정확도는 0.786


TF-IDF가 단순 카운트 기반보다 훨씬 높은 예측 정확도를 제공


텍스트 분석에서 머신러닝 모델의 성능을 향상시키는 2가지 방법
- ML  알고리즘을 선택하는 것
- 최상의 피처 전처리를 수행하는 것 

## 좀 더 다양한 파라미터를 적용한 TF-IDF
- 스톱 워드를  기존 'None'에서 'english'로,
- ngram_range는 기존 (1,1)에서 (1,2)로,
- max_df= 300 으로 변경

In [11]:
# stop words 필터링을 추가하고 ngram 을 기본 (1,1)에서 (1,2)로 변경해 피처 벡터화 적용
tfidf_vect = TfidfVectorizer(stop_words = 'english',ngram_range=(1,2),max_df=300)
tfidf_vect.fit(X_train,y_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect,y_train)

pred = lr_clf.predict(X_test_tfidf_vect)
print('CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(X_target,
                                                                                   pred)))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


CountVectorized Logistic Regression의 예측 정확도는 0.772


GridSearchCV를 이용해 로지스틱 회귀의 하이퍼 파라미터 최적화를 수행
 - 로지스틱 회귀의 C 파라미터만 변경하면서 최적의 C값을 찾은 뒤 이 C값으로 학습된 모델에서 테스트 데이터로 예측해 성능 평가

In [12]:
from sklearn.model_selection import GridSearchCV

# 최적 C 값 도출 튜닝 수행. CV는 3 폴드 세트로 설정
params ={'C':[0.01,0.1,1,5,10]}
grid_cv_lr = GridSearchCV(lr_clf,param_grid=params,cv=3,scoring='accuracy',verbose=1)
grid_cv_lr.fit(X_train_tfidf_vect,y_train)
print('Logistic Regression best C parameter:',grid_cv_lr.best_params_)

# 최적 C 값으로 학습된 grid_cv로 예측 및 정확도 평가
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:3f}'.format(accuracy_score(X_target,pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  5.3min finished


Logistic Regression best C parameter: {'C': 10}
TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.796734


GridSearchCV이 교차 검증 테스트 세트에서 가장 좋은 예측 성능을 나타냄.

# 사이킷런 파이프라인 사용  및 GridSearchCV와의 결합
pipeline = Pipeline([('tfidf_vect',TfidfVectorizer(stop_words='english')),
                     ('lr_clf',LogisticRegression(random_state=156))])

In [17]:
from sklearn.pipeline import Pipeline

#TfidVectorizer 객체를 tfidf_vect로, LogisticRegression객체를 lr_clf로 생성하는 Pipeline 새성
pipeline = Pipeline([
                     ('tfidf_vect',TfidfVectorizer(stop_words='english',ngram_range=(1,2),max_df=300)),
                     ('lr_clf',LogisticRegression(C=10))
])

#별도의 TfidfVectorizer 객체를 fit(), transform()과 LogisticRegression의 fit(), predict()가 필요없음.
#pipeline의 fit()과 predict()만으로 한꺼번에 피처 벡터화와 ML 학습/예측이 가능.
pipeline.fit(X_train,y_train)
pred = pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(
    accuracy_score(X_target,pred)))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Pipeline을 통한 Logistic Regression의 예측 정확도는 0.797
